In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
from swibrid.plot_clustering import *
from argparse import Namespace

In [2]:
args=Namespace(switch_coords='chr14:105583000-105872000:-',
               switch_annotation='hg38_switch_regions.bed',
               cutoff_color='r',
               color_by='meta_cluster2',
               chunksize=5000,
               variants_matrix=None,
               info='dummy',
               cmax=.1)

In [3]:
    import matplotlib
    import scipy.sparse
    from matplotlib import pyplot as plt
    from logzero import logger
    from swibrid.utils import (
        parse_switch_coords,
        read_switch_anno,
        get_switch_coverage,
        shift_coord,
    )

    matplotlib.rcParams.update({"font.size": 8})
    matplotlib.rcParams.update({"axes.linewidth": 0.5})
    matplotlib.rcParams.update({"xtick.major.width": 0.5})
    matplotlib.rcParams.update({"ytick.major.width": 0.5})

    # increase recursion limit for plotting large dendrograms
    sys.setrecursionlimit(100000)

    scale_bar_x_length = 5000
    scale_bar_x_legend = "5kb"

    (
        switch_chrom,
        switch_start,
        switch_end,
        switch_orientation,
    ) = parse_switch_coords(args.switch_coords)
    switch_anno = read_switch_anno(args.switch_annotation)
    cov_int, Ltot, eff_start, eff_end, anno_recs = get_switch_coverage(
        switch_anno, switch_chrom, switch_start, switch_end
    )

[I 251113 10:37:39 utils:53] using switch annotation from hg38_switch_regions.bed


In [4]:
def my_plot_main_image(args, msa, ax, Ltot, reads, clustering, order, values, cmap, variants=None, vmat=None):
    """plot the image in chunks"""
    import numpy as np
    import pandas as pd
    import matplotlib
    import re
    from matplotlib import pyplot as plt

    nreads = len(reads)

    nchunks = np.ceil(nreads / args.chunksize).astype(int)
    for n in range(nchunks):
        order_chunk = order[n * args.chunksize : min((n + 1) * args.chunksize, nreads)]
        extent = [
            0,
            Ltot,
            nreads - min((n + 1) * args.chunksize, nreads),
            nreads - n * args.chunksize,
        ]
        msa_chunk = msa[order_chunk]
        im = np.empty(msa_chunk.shape)
        im[:] = np.nan
        tmp = np.broadcast_to(values[order_chunk], msa_chunk.T.shape).T
        im[np.nonzero(msa_chunk)] = tmp[np.nonzero(msa_chunk)]

        ax.imshow(
            im,
            aspect="auto",
            interpolation="nearest",
            cmap=cmap,
            extent=extent,
            vmin=0,
            vmax=19
        ) 



In [5]:
def my_plot_inserts(args, cov_int, Ltot, ax, lw, nreads, order, clustering, switch_orientation):
    """plot insert locations"""
    import numpy as np
    import pandas as pd
    import re
    from collections import defaultdict
    from matplotlib.collections import LineCollection
    from swibrid.utils import (
        decode_insert,
        interval_length,
        intersect_intervals,
        merge_intervals,
        shift_coord,
    )

    assert args.coords is not None, "need processed read coordinates to show inserts!"

    process = pd.read_csv(args.coords, sep="\t", header=0, index_col=0)
    read_inserts = dict(
        (read, [decode_insert(insert) for insert in inserts.split(";")])
        for read, inserts in process["inserts"].dropna().items()
    )

    inserts = [
        (
            m.group("insert_chrom"),
            int(m.group("insert_start")),
            int(m.group("insert_end")),
        )
        for read, inserts in read_inserts.items()
        for m in inserts
    ]
    ninserts = len(inserts)
    unique_inserts = merge_intervals(inserts)
    ninserts_unique = len(unique_inserts)
    nclusts = len(np.unique(clustering["cluster"].astype(int)))
    nclusts_eff = len(
        np.unique(clustering["filtered_cluster"][clustering["filtered_cluster"] >= 0])
    )

    dx = 0.01 * Ltot

    for read in clustering.iloc[order].dropna(subset=["inserts"]).index:
        if read not in read_inserts:
            continue
        p = nreads - clustering.index[order].get_loc(read) - 1
        for m in read_inserts[read]:
            insert_chrom = m.group("insert_chrom")
            insert_start = int(m.group("insert_start"))
            insert_end = int(m.group("insert_end"))
            switch_left = int(m.group("switch_left"))
            switch_right = int(m.group("switch_right"))
            if (switch_orientation == "+" and switch_right < switch_left) or (
                switch_orientation == "-" and switch_right > switch_left
            ):
                switch_right, switch_left = switch_left, switch_right
            insert = (insert_chrom, insert_start, insert_end)
            ax.plot(
                shift_coord(switch_left, cov_int),
                p + 0.5,
                ">",
                color="k",
                markersize=0.5,
            )
            ax.plot(
                shift_coord(switch_right, cov_int),
                p + 0.5,
                "<",
                color="k",
                markersize=0.5,
            )
            
    return 

In [50]:
def my_plot_read_alignments(args, cov_int, Ltot, ax, fig, lw, nreads, mark_reads, mark_labels, mark_colors, order, clustering, realignments, switch_orientation):
    """plot insert locations"""
    import numpy as np
    import pandas as pd
    import re
    from collections import defaultdict
    from matplotlib.collections import LineCollection
    #from adjustText import adjust_text    
    from swibrid.utils import (
        decode_insert,
        interval_length,
        intersect_intervals,
        merge_intervals,
        shift_coord,
    )

    nlocs = len(mark_reads)
    dx = 0.01 * Ltot

    mark_pos = {}

    arrows = []
    texts = []
    all_labels = []
    for k, (read,label) in enumerate(zip(mark_reads,mark_labels)):
        p = nreads - clustering.index[order].get_loc(read) - 1
        if switch_orientation == "+":
            arrows.append([(Ltot, p + 0.5), (Ltot + dx, p + 0.5)])          
        else:
            arrows.append([(0, p + 0.5), (-dx, p + 0.5)])

        labels = [f"{label} {read}"]
        for _,x in realignments.loc[[read]].sort_values("pos_left", ascending=False).iterrows():
            if x['type']=='switch':
                labels.append('chr14:{3}-{4}: n_homology={0}, n_untemplated={1}'.format(x['n_homology'],x['n_untemplated'], k+1, 
                                                                                        x['pos_left'].split(':')[1],
                                                                                        x['pos_right'].split(':')[1]))
            else:
                labels.append('{3}-{4}'.format(x['n_homology'],x['n_untemplated'], k+1, 
                                               x['pos_left'],
                                               x['pos_right']))
            labels.append('\n'.join(x[['left_seq', 'match_left', 'read_seq', 'match_right','right_seq']]).upper())

        texts.append(ax.text(
            -4.2 * dx,
            p + 0.5,
            '\n'.join(labels),
            size="xx-small",
            color="k",
            clip_on=False,
            ha="left",
            va="center",
            family='monospace',
            bbox=dict(facecolor='none', edgecolor=mark_colors[k], pad=1.0, linewidth=1)
        ))

        ax.add_patch(matplotlib.patches.Rectangle((0, p + .5 -10), Ltot, 20, linewidth=lw, edgecolor='gray', facecolor='none'))

        all_labels.append(labels)


    transf = ax.transData.inverted()
    text_coords=[t.get_window_extent(renderer = fig.canvas.get_renderer()).transformed(transf).corners() for t in texts]
    text_heights=[tc[1][1]-tc[0][1] for tc in text_coords]
    sp=(np.diff(ax.get_ylim())[0]-sum(text_heights))/len(text_heights)
    assert sp > 0, 'negative space when adjusting labels'
        
    arrows=[]
    newp=ax.get_ylim()[1] 
    for k, t in enumerate(texts):
        x, p = t.get_position()
        newp -= .5*text_heights[k]
        t.set_position((x, newp))
        arrows.append([(-dx, p), (-3 * dx, newp)])
        arrows.append([(-3 * dx, newp), (-4 * dx, newp)])
        newp -= .5*text_heights[k] + sp 

    ax.add_collection(LineCollection(arrows, linewidths=0.25, colors="k", clip_on=False))

    return all_labels

# triplicates with 50000 cells of donor 21084 (=A in the paper)

In [51]:
donor='21084'
np.random.seed(0)

In [52]:
meta_clustering=pd.read_csv(f'meta_clustering/{donor}_50000_meta_clustering.csv',header=0,index_col=0)

In [53]:
df=meta_clustering.groupby('meta_cluster').apply(lambda df: pd.Series({'nreads_tot': df.shape[0], 'n_samples': len(df['sample'].unique()), 
                                                                       'min_reads_per_sample': df.groupby('sample').size().min(),
                                                                      'samples':','.join(df['sample'].unique())})).sort_values('nreads_tot')

In [54]:
top_clusters=df[df['n_samples']==3].iloc[-15:]
top_clusters

,nreads_tot,n_samples,min_reads_per_sample,samples
meta_cluster,,,,
458,213,3,36,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."
1126,213,3,50,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."
2137,214,3,56,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."
267,217,3,49,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."
296,224,3,65,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."
559,234,3,59,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."
176,242,3,47,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."
399,251,3,50,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."
450,259,3,60,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."


In [55]:
samples=top_clusters['samples'].unique()[0].split(",")

In [56]:
clustering = dict((sample,pd.read_csv(f'pipeline/{sample}/{sample}_clustering.csv', index_col=0, header=0)) for sample in samples)
realignments = dict((sample,pd.read_csv(f'pipeline/{sample}/{sample}_breakpoint_alignments.csv',header=0,index_col=0)) for sample in samples)

In [57]:
for sample in samples:
    realignments[sample]["chrom_left"] = realignments[sample]["pos_left"].str.split(":").str[0]
    realignments[sample]["chrom_right"] = realignments[sample]["pos_right"].str.split(":").str[0]
    realignments[sample]["pleft"] = realignments[sample]["pos_left"].str.split(":").str[1].astype(int)
    realignments[sample]["pright"] = realignments[sample]["pos_right"].str.split(":").str[1].astype(int)
    
    # filter out realignments across breaks smaller than max_realignment_gap      
    gap_size = np.abs(realignments[sample]["pleft"] - realignments[sample]["pright"])
    keep = (gap_size >= 75) | (realignments[sample]["chrom_left"] != realignments[sample]["chrom_right"])
    realignments[sample] = realignments[sample][keep]

In [58]:
coords = pd.concat([pd.read_csv(f'pipeline/{sample}/{sample}_processed.out', index_col=0, header=0, sep='\t').loc[clustering[sample].index].dropna() for sample in samples], axis=0, keys=samples)

In [59]:
from swibrid.utils import (
        decode_insert,
        interval_length,
        intersect_intervals,
        merge_intervals,
        shift_coord,
    )
read_inserts = dict((read, [decode_insert(insert) for insert in inserts.split(";")])
        for read, inserts in coords["inserts"].dropna().items())
inserts = [(m.group("insert_chrom"),int(m.group("insert_start")), int(m.group("insert_end")))
           for read, inserts in read_inserts.items()for m in inserts ]

In [60]:
unique_inserts = merge_intervals(inserts)

In [61]:
coords['uinsert']=[','.join(['{0}:{1}-{2}'.format(*ins) for ins in unique_inserts if 
                             interval_length(intersect_intervals([(m.group("insert_chrom"),
                                                                   int(m.group("insert_start")), 
                                                                   int(m.group("insert_end")))], [ins])) > 0][0] for m in inserts) for read, inserts in read_inserts.items()]

In [62]:
df2=coords.reset_index(names=['sample','read']).groupby('uinsert').apply(lambda df: pd.Series({'nreads_tot': df.shape[0], 'n_samples': len(df['sample'].unique()), 
                                                                       'min_reads_per_sample': df.groupby('sample').size().min(),
                                                                      'samples':','.join(df['sample'].unique())})).sort_values('nreads_tot')

In [63]:
top_inserts=df2[df2['n_samples'] == 3].iloc[-10:]
top_inserts

,nreads_tot,n_samples,min_reads_per_sample,samples
uinsert,,,,
chr16:22336035-22336493,6,3,1,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."
chr12:92467074-92467472,7,3,2,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."
chr3:141391307-141391556,21,3,2,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."


In [64]:
df3=meta_clustering.loc[coords[coords['uinsert'].isin(top_inserts.index)].index.get_level_values(1)][['meta_cluster','sample']].groupby('meta_cluster').apply(lambda df: pd.Series({'nreads_tot': df.shape[0], 'n_samples': len(df['sample'].unique()), 
                                                                       'min_reads_per_sample': df.groupby('sample').size().min(),
                                                                      'samples':','.join(df['sample'].unique())})).sort_values('nreads_tot')
df3[df3['n_samples']==3]

,nreads_tot,n_samples,min_reads_per_sample,samples
meta_cluster,,,,
10317,6,3,1,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."
659,11,3,2,"20211102_84_n50000_1,20211102_84_n50000_2,2021..."


In [65]:
top_insert_reads=meta_clustering.index[meta_clustering['meta_cluster'].isin([10317,659])]

In [66]:
selected_insert_reads={}
for sample in samples:
    selected_insert_reads[sample]=coords.loc[sample][coords.loc[sample]['uinsert'].isin(top_inserts.index) & coords.loc[sample].index.isin(top_insert_reads)]["uinsert"].drop_duplicates().index
    coords.loc[sample].loc[selected_insert_reads[sample]].to_csv(f'meta_clustering/{sample}_processed_filtered.out', sep='\t', index=True)

In [67]:
selected_clusters=list(top_clusters.index) 

In [68]:
figs=[plt.figure(figsize=(9,13)) for sample in samples]
lw = figs[0].bbox_inches.height * 72 / 50000

<Figure size 900x1300 with 0 Axes>

<Figure size 900x1300 with 0 Axes>

<Figure size 900x1300 with 0 Axes>

In [69]:
label_text={}

In [70]:
cmap=plt.cm.tab20b
cmap.set_under('#EEEEEE')
cat_type = pd.CategoricalDtype(categories=np.arange(len(selected_clusters)).astype(str), ordered=True)
cluster_map=dict(zip(np.array(selected_clusters).astype(str),np.arange(len(selected_clusters)).astype(str)))
order={}

for n, sample in enumerate(samples):

    ax=figs[n].add_axes([.005,.01,.015,.975])
    args.linkage=f'pipeline/{sample}/{sample}_linkage.npz'
    order[sample]=plot_linkage(args, clustering[sample], clustering[sample]['cluster'].dropna().shape[0], ax, .1, lw)

    ax=figs[n].add_axes([.02,.01,.4,.975])
    msa=scipy.sparse.load_npz(f'pipeline/{sample}/{sample}_msa.npz').tocsr()
    ri=meta_clustering[meta_clustering['sample']==sample].loc[clustering[sample].index]
    ri[~ri['meta_cluster'].isin(selected_clusters)]['meta_cluster']=np.nan
    ri['meta_cluster2']=ri['meta_cluster'].astype(str).map(cluster_map).astype(str).astype(cat_type)
    values=ri['meta_cluster2'].cat.codes.values

    variants = pd.read_csv(f'pipeline/{sample}/{sample}_variants.txt', sep="\t", header=0)
    vmat = scipy.sparse.load_npz(f'pipeline/{sample}/{sample}_variants.npz').tocsr()

    my_plot_main_image(args, msa, ax, Ltot, clustering[sample]['cluster'].dropna().index, 
                       clustering[sample], order[sample], values, cmap, variants, vmat)
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_yticks([])
    ax.set_xticks([])
    xlim = [0, Ltot]
    ylim = [0, msa.shape[0]]
    ax.set_xlim(xlim if switch_orientation == "+" else xlim[::-1])
    ax.set_ylim(ylim)

    major_ticks = []
    minor_ticks = []
    minor_labels = []
    for rec in anno_recs:
        start = shift_coord(int(rec[3][1]), cov_int)
        end = shift_coord(int(rec[3][2]), cov_int)
        major_ticks += [start, end]
        minor_ticks.append((start + end) / 2)
        minor_labels.append(rec[3][3])
    ax.set_xticks(np.unique(major_ticks))
    ax.set_xticklabels([])
    ax.set_xticks(minor_ticks, minor=True)
    ax.set_xticklabels(minor_labels, minor=True, size="small")
    ax.tick_params(which="minor", length=0)
    
    args.coords=f'meta_clustering/{sample}_processed_filtered.out'
    my_plot_inserts(args, cov_int, Ltot, ax, lw, msa.shape[0], order[sample], clustering[sample], switch_orientation)

    selected_reads=np.array([np.random.choice(meta_clustering[(meta_clustering['meta_cluster']==c) & (meta_clustering['sample']==sample)].index, size=1)[0] for c in top_clusters.index] + list(selected_insert_reads[sample]))
    selected_labels=np.array([f'#{k+1}' for k in range(len(selected_reads))])
    selected_colors=np.array([cmap(int(cluster_map[str(c)])) for c in selected_clusters] + [(0,0,0,1) for i in selected_insert_reads[sample]])
    selected_order=pd.Series(dict((read,clustering[sample].index[order[sample]].get_loc(read)) for read in selected_reads)).argsort()
    label_text[sample] = my_plot_read_alignments(args, cov_int, Ltot, ax, figs[n], lw, msa.shape[0], selected_reads[selected_order], selected_labels[selected_order],
                            selected_colors[selected_order], order[sample], clustering[sample], realignments[sample], switch_orientation)

/data/cephfs-1/home/users/obermayb_c/scratch/tmp/ipykernel_2428114/465303758.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ri[~ri['meta_cluster'].isin(selected_clusters)]['meta_cluster']=np.nan
/data/cephfs-1/home/users/obermayb_c/scratch/tmp/ipykernel_2428114/465303758.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ri[~ri['meta_cluster'].isin(selected_clusters)]['meta_cluster']=np.nan
/data/cephfs-1/home/users/obermayb_c/scratch/tmp/ipykernel_2428114/465303758.py:16: SettingWithCopyWarning: 
A

In [29]:
for n,sample in enumerate(samples):
    replicate=n+1
    figs[n].text(0.01, 0.99, f'top recurring clusters in replicate {replicate} of 50000 B cells of donor A', 
                 size="small", ha="left", va="bottom")
    figs[n].savefig(f'meta_clustering/{sample}_cluster_tracing.pdf' , dpi=500)

In [27]:
plt.close('all')

# comparison of pacBio and minION reads from the same sample

In [71]:
donor='HD19005'
meta_clustering=pd.read_csv(f'meta_clustering/{donor}_meta_clustering.csv',header=0,index_col=0)
df=meta_clustering.groupby('meta_cluster').apply(lambda df: pd.Series({'nreads_tot': df.shape[0], 'n_samples': len(df['sample'].unique()), 
                                                                       'min_reads_per_sample': df.groupby('sample').size().min(),
                                                                      'samples':','.join(df['sample'].unique())})).sort_values('nreads_tot')
top_clusters=df[df['n_samples']==2].iloc[-20:]
samples=top_clusters['samples'].unique()[0].split(",")
top_clusters

,nreads_tot,n_samples,min_reads_per_sample,samples
meta_cluster,,,,
22,124,2,31,"20230313_HD19005,20230504_HD19005"
0,125,2,49,"20230313_HD19005,20230504_HD19005"
143,125,2,14,"20230313_HD19005,20230504_HD19005"
120,125,2,37,"20230313_HD19005,20230504_HD19005"
11,127,2,40,"20230313_HD19005,20230504_HD19005"
19,132,2,45,"20230313_HD19005,20230504_HD19005"
48,138,2,42,"20230313_HD19005,20230504_HD19005"
39,139,2,19,"20230313_HD19005,20230504_HD19005"
82,143,2,40,"20230313_HD19005,20230504_HD19005"


In [72]:
selected_clusters=list(top_clusters.index) 

In [73]:
clustering = dict((sample,pd.read_csv(f'pipeline/{sample}/{sample}_clustering.csv', index_col=0, header=0)) for sample in samples)
realignments = dict((sample,pd.read_csv(f'pipeline/{sample}/{sample}_breakpoint_alignments.csv',header=0,index_col=0)) for sample in samples)

In [74]:
for sample in samples:
    realignments[sample]["chrom_left"] = realignments[sample]["pos_left"].str.split(":").str[0]
    realignments[sample]["chrom_right"] = realignments[sample]["pos_right"].str.split(":").str[0]
    realignments[sample]["pleft"] = realignments[sample]["pos_left"].str.split(":").str[1].astype(int)
    realignments[sample]["pright"] = realignments[sample]["pos_right"].str.split(":").str[1].astype(int)
    
    # filter out realignments across breaks smaller than max_realignment_gap      
    gap_size = np.abs(realignments[sample]["pleft"] - realignments[sample]["pright"])
    keep = (gap_size >= 75) | (realignments[sample]["chrom_left"] != realignments[sample]["chrom_right"])
    realignments[sample] = realignments[sample][keep]

In [75]:
figs=[plt.figure(figsize=(9,13)) for sample in samples]
lw = figs[0].bbox_inches.height * 72 / 50000

<Figure size 900x1300 with 0 Axes>

<Figure size 900x1300 with 0 Axes>

In [76]:
cmap=plt.cm.tab20b
cmap.set_under('#EEEEEE')
cat_type = pd.CategoricalDtype(categories=np.arange(len(selected_clusters)).astype(str), ordered=True)
cluster_map=dict(zip(np.array(selected_clusters).astype(str),np.arange(len(selected_clusters)).astype(str)))
order={}

for n, sample in enumerate(samples):

    ax=figs[n].add_axes([.005,.01,.015,.975])
    args.linkage=f'pipeline/{sample}/{sample}_linkage.npz'
    order[sample]=plot_linkage(args, clustering[sample], clustering[sample]['cluster'].dropna().shape[0], ax, .1, lw)

    ax=figs[n].add_axes([.02,.01,.6,.975])
    msa=scipy.sparse.load_npz(f'pipeline/{sample}/{sample}_msa.npz').tocsr()
    ri=meta_clustering[meta_clustering['sample']==sample].loc[clustering[sample].index]
    ri[~ri['meta_cluster'].isin(selected_clusters)]['meta_cluster']=np.nan
    ri['meta_cluster2']=ri['meta_cluster'].astype(str).map(cluster_map).astype(str).astype(cat_type)
    values=ri['meta_cluster2'].cat.codes.values

    variants = pd.read_csv(f'pipeline/{sample}/{sample}_variants.txt', sep="\t", header=0)
    vmat = scipy.sparse.load_npz(f'pipeline/{sample}/{sample}_variants.npz').tocsr()

    my_plot_main_image(args, msa, ax, Ltot, clustering[sample]['cluster'].dropna().index, 
                       clustering[sample], order[sample], values, cmap, variants, vmat)
    ax.spines["top"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_yticks([])
    ax.set_xticks([])
    xlim = [0, Ltot]
    ylim = [0, msa.shape[0]]
    ax.set_xlim(xlim if switch_orientation == "+" else xlim[::-1])
    ax.set_ylim(ylim)

    major_ticks = []
    minor_ticks = []
    minor_labels = []
    for rec in anno_recs:
        start = shift_coord(int(rec[3][1]), cov_int)
        end = shift_coord(int(rec[3][2]), cov_int)
        major_ticks += [start, end]
        minor_ticks.append((start + end) / 2)
        minor_labels.append(rec[3][3])
    ax.set_xticks(np.unique(major_ticks))
    ax.set_xticklabels([])
    ax.set_xticks(minor_ticks, minor=True)
    ax.set_xticklabels(minor_labels, minor=True, size="small")
    ax.tick_params(which="minor", length=0)
    
    selected_reads=np.array([np.random.choice(meta_clustering[(meta_clustering['meta_cluster']==c) & (meta_clustering['sample']==sample)].index, size=1)[0] for c in top_clusters.index])
    selected_labels=np.array([f'#{k+1}' for k in range(top_clusters.shape[0])])
    selected_colors=np.array([cmap(int(cluster_map[str(c)])) for c in top_clusters.index])
    selected_order=pd.Series(dict((read,clustering[sample].index[order[sample]].get_loc(read)) for read in selected_reads)).argsort()
    label_text[sample]=my_plot_read_alignments(args, cov_int, Ltot, ax, figs[n], lw, msa.shape[0], selected_reads[selected_order], 
                            selected_labels[selected_order], selected_colors[selected_order], order[sample], clustering[sample], realignments[sample], switch_orientation)

/data/cephfs-1/home/users/obermayb_c/scratch/tmp/ipykernel_2428114/2912412467.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ri[~ri['meta_cluster'].isin(selected_clusters)]['meta_cluster']=np.nan
/data/cephfs-1/home/users/obermayb_c/scratch/tmp/ipykernel_2428114/2912412467.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ri[~ri['meta_cluster'].isin(selected_clusters)]['meta_cluster']=np.nan


In [37]:
for n,sample in enumerate(samples):
    method='minION' if sample.startswith('20230313') else 'pacBio'
    figs[n].text(0.01, 0.99, f'top recurring clusters for PBMCs ({method} sequencing)', 
                 size="small", ha="left", va="bottom")
    figs[n].savefig(f'meta_clustering/{sample}_cluster_tracing.pdf' , dpi=500)

In [35]:
plt.close('all')

In [80]:
with pd.ExcelWriter("cluster_tracing.xlsx") as writer:
    for n,sample in enumerate(['20211102_84_n50000_1', '20211102_84_n50000_2', '20211122_84_n50000_3']):
        replicate=n+1
        sheet=f'B cells of donor A (R{replicate})'
        pd.DataFrame(label_text[sample]).to_excel(writer, sheet_name=sheet, header=False, index=False)
    for n,sample in enumerate(['20230313_HD19005', '20230504_HD19005']):
        method='minION' if sample.startswith('20230313') else 'pacBio'
        sheet=f'{method} sequencing of PBMCs'
        pd.DataFrame(label_text[sample]).to_excel(writer, sheet_name=sheet, header=False, index=False)